<a href="https://colab.research.google.com/github/zc0125/colab-stable-diffusion-webui/blob/main/v1/run_stable_diffusion_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 运行前同步运行资源到运行目录
# 进行生成任务时，可以尝试以下几个方面来获得更好的结果：

# 1. 调整模型参数：stable_diffusion 提供了多个可以调整的参数，如扰动系数、迭代次数等等。根据具体任务的需要，可以进行适当调整。

# 2. 使用更大的模型：如果任务需要更高的生成质量或更高的分辨率，可以考虑使用更大的模型。但需要注意，使用更大的模型会占用更多的显存，可能需要更高的计算资源。

# 3. 优化输入数据：稳定扰动生成模型对输入数据质量要求较高。可以通过对输入数据进行预处理、调整分辨率等操作，提高输入数据的质量，进而提高生成结果的质量。

# 4. 使用多个模型进行集成：可以尝试将多个模型的生成结果进行集成，获得更好的生成效果。集成方法可以是简单的平均，也可以使用更复杂的加权平均等方法。

import os
from google.colab.output import eval_js
from google.colab import drive
%cd /content/
drive.mount('/content/drive')
os.environ['colab_url'] = eval_js("google.colab.kernel.proxyPort(7860, {'cache': false})")

!apt -y update -qq
!wget http://launchpadlibrarian.net/367274644/libgoogle-perftools-dev_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/google-perftools_2.5-2.2ubuntu3_all.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libtcmalloc-minimal4_2.5-2.2ubuntu3_amd64.deb
!wget https://launchpad.net/ubuntu/+source/google-perftools/2.5-2.2ubuntu3/+build/14795286/+files/libgoogle-perftools4_2.5-2.2ubuntu3_amd64.deb
!apt install -qq libunwind8-dev
!dpkg -i *.deb
%env LD_PRELOAD=libtcmalloc.so
!rm *.deb
!apt -y install -qq aria2

uninstall_flag = True
try:
  import torch
  uninstall_flag = False if torch.__version__ == "1.13.1+cu117" else True
except:
  pass
if uninstall_flag:
# 冲突卸载重装
  !pip uninstall torchvision -y
  !pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 --extra-index-url https://download.pytorch.org/whl/cu117
!pip uninstall xformers -y
!CUDA=1 pip install xformers==0.0.16rc425
!pip install -q --pre triton==2.0.0 -U

# 同步stable-diffusion-webui项目
!test -d /content/stable-diffusion-webui || git clone https://github.com/AUTOMATIC1111/stable-diffusion-webui.git /content/stable-diffusion-webui
%cd /content/stable-diffusion-webui/
# 如果新版本有问题重置到稳定版本
# !git reset --hard 22bcc7be428c94e9408f589966c2040187245d81
# 初始化检查文件夹
!test -d /content/stable-diffusion-webui/extensions || mkdir /content/stable-diffusion-webui/extensions -p
!test -d /content/stable-diffusion-webui/models/Stable-diffusion || mkdir /content/stable-diffusion-webui/models/Stable-diffusion -p
!test -d /content/stable-diffusion-webui/models/Lora || mkdir /content/stable-diffusion-webui/models/Lora -p
!test -d /content/stable-diffusion-webui/models/ControlNet || mkdir /content/stable-diffusion-webui/models/ControlNet -p
!test -d /content/stable-diffusion-webui/models/BLIP || mkdir /content/stable-diffusion-webui/models/BLIP -p
!test -d /content/stable-diffusion-webui/models/Codeformer || mkdir /content/stable-diffusion-webui/models/Codeformer -p
# 同步并使用网盘上的图片保存路径（如果网盘存在对应的路径）
!test -d /content/drive/MyDrive/stable-diffusion-webui/outputs || mv /content/stable-diffusion-webui/outputs /content/drive/MyDrive/stable-diffusion-webui/
!test -d /content/drive/MyDrive/stable-diffusion-webui/outputs && test -d /content/stable-diffusion-webui/outputs && rm -rf /content/stable-diffusion-webui/outputs
!test -d /content/drive/MyDrive/stable-diffusion-webui/outputs && ln -s /content/drive/MyDrive/stable-diffusion-webui/outputs /content/stable-diffusion-webui/outputs
# 同步并使用网盘上的models（如果网盘存在对应的路径）
# 先同步之前的备份
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_Stable-diffusion && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/back_Stable-diffusion)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/back_Stable-diffusion/* /content/stable-diffusion-webui/models/Stable-diffusion/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_Lora && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/back_Lora)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/back_Lora/* /content/stable-diffusion-webui/models/Lora/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_ControlNet && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/back_ControlNet)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/back_ControlNet/* /content/stable-diffusion-webui/models/ControlNet/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_BLIP && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/back_BLIP)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/back_BLIP/* /content/stable-diffusion-webui/models/BLIP/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_Codeformer && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/back_Codeformer)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/back_Codeformer/* /content/stable-diffusion-webui/models/Codeformer/)
# 新的model覆盖备份
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/share_stable-diffusion*/* /content/stable-diffusion-webui/models/Stable-diffusion/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Lora && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/Lora)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/share_lora*/* /content/stable-diffusion-webui/models/Lora/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet*/* /content/stable-diffusion-webui/models/ControlNet/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/BLIP && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/BLIP)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/BLIP*/* /content/stable-diffusion-webui/models/BLIP/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer*/* /content/stable-diffusion-webui/models/Codeformer/)

# 复制model到运行路径 （目前使用的是软链）
# !test -d /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion && cp /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion/* /content/stable-diffusion-webui/models/Stable-diffusion/ -ru
# !test -d /content/drive/MyDrive/stable-diffusion-webui/models/Lora && cp /content/drive/MyDrive/stable-diffusion-webui/models/Lora/* /content/stable-diffusion-webui/models/Lora/ -ru
# !test -d /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet && cp /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet/* /content/stable-diffusion-webui/models/ControlNet/ -ru
# !test -d /content/drive/MyDrive/stable-diffusion-webui/models/BLIP && cp /content/drive/MyDrive/stable-diffusion-webui/models/BLIP/* /content/stable-diffusion-webui/models/BLIP/ -ru
# !test -d /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer && cp /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer/* /content/stable-diffusion-webui/models/Codeformer/ -ru

# 同步并使用网盘上的拓展（如果网盘存在对应的路径）（有的拓展同步运行存在问题不要使用）
# !test -d /content/drive/MyDrive/stable-diffusion-webui/extensions || mv /content/stable-diffusion-webui/extensions /content/drive/MyDrive/stable-diffusion-webui/
# !test -d /content/drive/MyDrive/stable-diffusion-webui/extensions && test -d /content/stable-diffusion-webui/extensions && rm -rf /content/stable-diffusion-webui/extensions
# !ln -s /content/drive/MyDrive/stable-diffusion-webui/extensions /content/stable-diffusion-webui/extensions
# 复制拓展应该可以(能在网上下载的拓展尽量不要保存，复制起来很慢)
# !test -d /content/drive/MyDrive/stable-diffusion-webui/extensions && ln -s -f /content/drive/MyDrive/stable-diffusion-webui/extensions/* /content/stable-diffusion-webui/extensions
# !test -d /content/drive/MyDrive/stable-diffusion-webui/extensions && cp -r -u /content/drive/MyDrive/stable-diffusion-webui/extensions/* /content/stable-diffusion-webui/extensions
# 安装新的拓展
!test -d /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser || git clone https://github.com/AlUlkesh/stable-diffusion-webui-images-browser /content/stable-diffusion-webui/extensions/stable-diffusion-webui-images-browser
!test -d /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface || git clone https://github.com/camenduru/stable-diffusion-webui-huggingface /content/stable-diffusion-webui/extensions/stable-diffusion-webui-huggingface
!test -d /content/stable-diffusion-webui/extensions/sd-civitai-browser || git clone -b v2.0 https://github.com/camenduru/sd-civitai-browser /content/stable-diffusion-webui/extensions/sd-civitai-browser
!test -d /content/stable-diffusion-webui/extensions/sd-webui-additional-networks || git clone https://github.com/kohya-ss/sd-webui-additional-networks /content/stable-diffusion-webui/extensions/sd-webui-additional-networks
!test -d /content/stable-diffusion-webui/extensions/sd-webui-controlnet || git clone https://github.com/Mikubill/sd-webui-controlnet /content/stable-diffusion-webui/extensions/sd-webui-controlnet
!test -d /content/stable-diffusion-webui/extensions/openpose-editor || git clone https://github.com/camenduru/openpose-editor /content/stable-diffusion-webui/extensions/openpose-editor
!test -d /content/stable-diffusion-webui/extensions/sd-webui-depth-lib || git clone https://github.com/jexom/sd-webui-depth-lib /content/stable-diffusion-webui/extensions/sd-webui-depth-lib
!test -d /content/stable-diffusion-webui/extensions/posex || git clone https://github.com/hnmr293/posex /content/stable-diffusion-webui/extensions/posex
!test -d /content/stable-diffusion-webui/extensions/sd-webui-tunnels || git clone https://github.com/camenduru/sd-webui-tunnels /content/stable-diffusion-webui/extensions/sd-webui-tunnels
!test -d /content/stable-diffusion-webui/extensions/batchlinks-webui || git clone https://github.com/etherealxx/batchlinks-webui /content/stable-diffusion-webui/extensions/batchlinks-webui
!test -d /content/stable-diffusion-webui/extensions/stable-diffusion-webui-chinese || git clone https://github.com/VinsonLaro/stable-diffusion-webui-chinese /content/stable-diffusion-webui/extensions/stable-diffusion-webui-chinese
!test -d /content/stable-diffusion-webui/extensions/tagcomplete || git clone -b branch/chinese-ex https://github.com/zc0125/webui-tagcomplete.git /content/stable-diffusion-webui/extensions/tagcomplete
# 使用备份配置文件
!test -f /content/drive/MyDrive/stable-diffusion-webui/config.json && ln -sf /content/drive/MyDrive/stable-diffusion-webui/config.json /content/stable-diffusion-webui/config.json


# 直接启动
# !python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
!python launch.py --listen --xformers --enable-insecure-extension-access --theme dark --gradio-queue --multiple

In [ ]:
# 同步本次运行配置文件
!test -f /content/stable-diffusion-webui/config.json && cp /content/stable-diffusion-webui/config.json /content/drive/MyDrive/stable-diffusion-webui/config.json
!test -f /content/drive/MyDrive/stable-diffusion-webui/config.json && ln -sf /content/drive/MyDrive/stable-diffusion-webui/config.json /content/stable-diffusion-webui/config.json

In [ ]:
# 重置运行目录
%cd /content/
!rm -rf /content/stable-diffusion-webui

In [ ]:
# 如果网盘上不存在outputs文件夹，将outputs同步到网盘，以备下次使用
!test -d /content/drive/MyDrive/stable-diffusion-webui/outputs || mv /content/stable-diffusion-webui/outputs /content/drive/MyDrive/stable-diffusion-webui/
!test -d /content/drive/MyDrive/stable-diffusion-webui/outputs && test -d /content/stable-diffusion-webui/outputs && rm -rf /content/stable-diffusion-webui/outputs
!ln -s /content/drive/MyDrive/stable-diffusion-webui/outputs /content/stable-diffusion-webui/outputs

In [ ]:
# 如果网盘上不存在extensions文件夹，将extensions同步到网盘，以备下次使用（目前先不要使用，有的拓展存在问题）
# !test -d /content/drive/MyDrive/stable-diffusion-webui/extensions || mv /content/stable-diffusion-webui/extensions /content/drive/MyDrive/stable-diffusion-webui/
# !test -d /content/drive/MyDrive/stable-diffusion-webui/extensions && test -d /content/stable-diffusion-webui/extensions && rm -rf /content/stable-diffusion-webui/extensions
# !ln -s /content/drive/MyDrive/stable-diffusion-webui/extensions /content/stable-diffusion-webui/extensions

In [ ]:
# (如果网盘有足够的使用空间) 可以把当前新下载的model同步至网盘，以备下次使用
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_Stable-diffusion || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/back_Stable-diffusion -p
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_Lora || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/back_Lora -p
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_ControlNet || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/back_ControlNet -p
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_BLIP || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/models/back_BLIP -p
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/back_Codeformer || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/back_Codeformer -p
!test -d /content/stable-diffusion-webui/models/Stable-diffusion && cp -r -u /content/stable-diffusion-webui/models/Stable-diffusion/* /content/drive/MyDrive/stable-diffusion-webui/models/back_Stable-diffusion/
!test -d /content/stable-diffusion-webui/models/Lora && cp -r -u /content/stable-diffusion-webui/models/Lora/* /content/drive/MyDrive/stable-diffusion-webui/models/back_Lora/
!test -d /content/stable-diffusion-webui/models/ControlNet && cp -r -u /content/stable-diffusion-webui/models/ControlNet/* /content/drive/MyDrive/stable-diffusion-webui/models/back_ControlNet/
!test -d /content/stable-diffusion-webui/models/BLIP && cp -r -u /content/stable-diffusion-webui/models/BLIP/* /content/drive/MyDrive/stable-diffusion-webui/models/back_BLIP/
!test -d /content/stable-diffusion-webui/models/Codeformer && cp -r -u /content/stable-diffusion-webui/models/Codeformer/* /content/drive/MyDrive/stable-diffusion-webui/models/back_Codeformer/

In [ ]:
#重新挂载网盘
from google.colab import drive
%cd /content/
drive.mount('/content/drive')

In [ ]:
# 重新加载model，执行当前脚本将model同步到运行目录
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion*/* /content/stable-diffusion-webui/models/Stable-diffusion/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Lora && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/Lora)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/Lora*/* /content/stable-diffusion-webui/models/Lora/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet*/* /content/stable-diffusion-webui/models/ControlNet/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/BLIP && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/BLIP)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/BLIP*/* /content/stable-diffusion-webui/models/BLIP/)
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer && (test -z "$(ls -A /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer)" || ln -sf /content/drive/MyDrive/stable-diffusion-webui/models/Codeformer*/* /content/stable-diffusion-webui/models/Codeformer/)

In [ ]:
# 下载model模型
Stable_diffusion_list = [
    # "https://civitai.com/api/download/models/15236",
    # "https://civitai.com/api/download/models/24365",
    # "https://civitai.com/api/download/models/19084",

]
Lora_list = [
    # "https://civitai.com/api/download/models/27889",

]
ControlNet_list = [
    # "https://huggingface.co/webui/ControlNet-modules-safetensors/blob/main/control_canny-fp16.safetensors",
]

!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Lora || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/Lora -p
for _url in Lora_list:
  print(_url)
  !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 $_url -d /content/drive/MyDrive/stable-diffusion-webui/models/Lora
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion -p
for _url in Stable_diffusion_list:
  print(_url)
  !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 $_url -d /content/drive/MyDrive/stable-diffusion-webui/models/Stable-diffusion
!test -d /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet || mkdir /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet -p
for _url in Stable_diffusion_list:
  print(_url)
  !aria2c --console-log-level=error -c -x 16 -k 1M -s 16 $_url -d /content/drive/MyDrive/stable-diffusion-webui/models/ControlNet
